In [1]:
#!pip install transformers[sentencepiece]

In [2]:
#!pip install datasets

In [3]:
from datasets import load_dataset, load_metric

In [3]:
import os
import statistics
import s3fs
import pandas as pd

## Loading the model 

In [5]:
from transformers import pipeline

nlp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')


2023-04-25 14:04:51.713839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 14:04:52.779794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import collections

## définition des fonctions d'évaluation

In [1]:
def normalize_text(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return " ".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)
    def pointov(text):
        regex="\."
        return re.sub(regex,",",text)
    return white_space_fix(pointov(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [2]:
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

In [55]:
def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [58]:
def compute_acc(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    else :
        return 1

## Loading the data RPQS_125

In [10]:
from datasets import load_dataset
import json

In [11]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "mberthe/narval"
fs.ls(BUCKET)
BUCKET = "mberthe/narval"
FILE_KEY_S3 = "camemBERT/answers(2).json"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    f1=json.load(file_in)

In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [14]:
df=pd.DataFrame(f1["data"])
df.head()

,paragraphs
0,[{'qas': [{'question': 'quelle est la valeur ...
1,[{'qas': [{'question': 'quelle est la valeur d...
2,[{'qas': [{'question': 'Quelle est la valeur d...
3,[{'qas': [{'question': 'quelle est la valeur d...
4,[{'qas': [{'question': 'Quelle est la valeur d...


In [15]:
input_file_path = FILE_PATH_S3
record_path = ['data','paragraphs','qas','answers']
rpqs = pd.json_normalize(f1, record_path =record_path,meta=[
        ['data', 'paragraphs','context'], 
        ['data', 'paragraphs', 'qas','id'],
        ['data', 'paragraphs', 'qas','question']])

In [20]:
rpqs=rpqs.rename(columns={"data.paragraphs.context":"context","data.paragraphs.qas.id":"id","data.paragraphs.qas.question":"question"})

In [ ]:
pred=pd.DataFrame()
for i in rpqs.index:
    d=nlp({'question': rpqs["question"][i],
         'context': rpqs['context'][i]})
    #print(d)
    df1=pd.DataFrame(d,index=[i])
    pred=pd.concat([pred,df1])
    #pred=pred.append(df1)
print(pred.sample(15))
#pred["answer"]

## Evaluation

In [75]:
moyennes_score=statistics.mean(pred["score"])
print(moyennes_score)

0.4976845828769001


In [76]:
#print(data['text'])
print(pred['answer'])

0                                             \n• 99,8 %
1            \n• 99,8 % en terme de microbiologie• 100 %
2                                         entre 0 et 120
3                                         entre 0 et 120
4                                 P102.1/parenright.case
                             ...                        
121      2,11% pour les services exploités par \nla RESE
122                               0,20 pour 1000 abonnés
123     Taux de réclamations  \n  0,20 pour 1000 abonnés
124                                                    €
125                                             (P109.0)
Name: answer, Length: 126, dtype: object


### Exact Match

In [86]:
em_list=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    prediction=pred["answer"][i] 
    em=compute_exact_match(prediction,truth)
    em_list.append(em)
moyenne_em=statistics.mean(em_list)
print('em_moyen=',moyenne_em)

em_moyen= 0.10317460317460317


### F1

In [88]:
f1_list=[]
for i in rpqs.index:
    truth=rpqs["text"][i]
    prediction=pred["answer"][i] 
    f1=compute_f1(prediction,truth)
    #print("f1",f1)
    f1_list.append(f1)
#print(f1_list)
moyenne_f1=statistics.mean(f1_list)
print('f1_moyen=',moyenne_f1)

f1_moyen= 0.27272326438993105


### Accuracy

In [90]:
acc_list=[]
for i in range(0,len(rpqs)):
    truth=rpqs["text"][i]
    prediction=pred["answer"][i] 
    acc=compute_acc(prediction,truth)
    #print("acc",acc)
    acc_list.append(acc)
moyenne_acc=statistics.mean(acc_list)
print('acc_moyen=',moyenne_acc)

acc_moyen= 0.4523809523809524
